In [4]:
import ast

In [147]:
# fname = 'example.txt'
fname = 'input.txt'
with open(fname) as f:
    lines = f.readlines()
data = [ast.literal_eval(line.strip()) for line in lines]
data = [line.strip() for line in lines]

In [148]:
# # even smaller test
# data = ast.literal_eval("[[[[[9,8],1],2],3],4]")
# data = "[[[[[9,8],1],2],3],4]"

In [149]:
data

['[[[6,[8,3]],[2,0]],[[[9,5],[9,1]],3]]',
 '[[[9,[2,2]],[5,4]],[[[2,2],[9,6]],[7,7]]]',
 '[[[0,[3,2]],1],[[0,[2,8]],[2,[0,4]]]]',
 '[[4,4],[[[7,0],5],[3,1]]]',
 '[[5,4],1]',
 '[[[[7,6],4],9],[[9,1],9]]',
 '[[[1,[7,8]],[[7,7],[1,6]]],[1,[6,[7,1]]]]',
 '[[[[6,8],[5,6]],[[1,1],8]],[[[2,0],[3,1]],[2,[2,6]]]]',
 '[[[6,3],[3,[7,1]]],8]',
 '[[[9,4],[3,[0,6]]],[[2,[3,6]],[[9,8],[1,6]]]]',
 '[9,[0,[[0,7],2]]]',
 '[[[[8,4],7],[[9,2],[0,9]]],[[7,9],[8,[0,9]]]]',
 '[[1,1],[[5,[3,8]],[3,[4,7]]]]',
 '[[[9,[2,9]],[2,[2,9]]],[[[3,5],5],[[3,3],2]]]',
 '[[[[5,4],9],0],[[[5,7],2],[[5,2],9]]]',
 '[[2,[[1,0],[6,2]]],0]',
 '[[[3,7],[7,6]],[[[2,8],5],[3,[9,7]]]]',
 '[[2,[2,[8,8]]],[[[9,9],[1,1]],[[8,6],[0,3]]]]',
 '[[8,1],[3,5]]',
 '[[7,[[7,6],[2,0]]],4]',
 '[[5,4],[[1,3],[5,[2,8]]]]',
 '[7,9]',
 '[[[[6,9],0],[1,[5,0]]],[[[6,4],3],7]]',
 '[[[[3,7],3],[2,6]],[[0,4],[9,9]]]',
 '[[[[1,5],[5,0]],[9,4]],[[[8,3],3],[8,[3,6]]]]',
 '[[[[3,7],5],[[8,5],[1,5]]],[[0,6],[3,4]]]',
 '[[[[4,0],2],[7,[8,4]]],[0,[5,[7,8]]]]'

In [31]:
def explore(sf_num, depth=0):
    left, right = sf_num
    print(f"{depth}\tleft: {type(left)}\t right: {type(right)}")
    if depth >= 4:
        print("Explode me!")
    if type(left) == list:
        explore(left, depth=depth+1)
    if type(right) == list:
        explore(right, depth=depth+1)

# for num in data:
#     explore(num)

explore(data)

0	left: <class 'list'>	 right: <class 'int'>
1	left: <class 'list'>	 right: <class 'int'>
2	left: <class 'list'>	 right: <class 'int'>
3	left: <class 'list'>	 right: <class 'int'>
4	left: <class 'int'>	 right: <class 'int'>
Explode me!


In [51]:
def explode(sf_num, depth=0):
    left, right = sf_num
    print(f"{depth}\tleft: {type(left)}\t right: {type(right)}")
    if depth >= 4:
        print("Explode me!")
        return (left, right)
#         sf_num[0] += 1
    if type(left) == list:
        lr = explode(left, depth=depth+1)
        if lr:
            # try to add to right side, because child is on the left
            if type(right) == int:
                child_l, child_r = lr
                if child_r:
                    sf_num[1] += child_r
                else:
                    # already added somewhere
                    return ...
            else:
                # not available on the right
                return ...
        else: 
            return lr
    if type(right) == list:
        lr explode(right, depth=depth+1)
        if lr:
            # try to add to left side, because child is on the right
            pass
        else: 
            return lr

# for num in data:
#     explore(num)

explode(data)
data

SyntaxError: invalid syntax (Temp/ipykernel_7392/1669669707.py, line 25)

# alternate approach

In [49]:
# keep track of two lists (or a list of pairs) which have number and depth.
# adjacent numbers at the same depth are paired. 
# two numbers at depth 4 are elligible for explode
    # this can be found easily by depth list
    # explode can be carried out by referencing indices

In [130]:
def parse_sfn(sfn):
    values = []
    depths = []
    depth = 0
    for char in sfn:
        if char == '[':
            depth += 1
        elif char == ']':
            depth -= 1
        elif char == ',':
            continue
        else:
            values.append(int(char))
            depths.append(depth)
    return values, depths

In [110]:
def find_explode(depths):
    for ii, pair in enumerate(zip(depths, depths[1:])):
        if pair == (5, 5):
            return ii
    return None

def explode(values, depths, idx):
    # idx is the first of the pair
    left, right = values[idx], values[idx + 1]
#     print(f"{left=}, {right=}")
    # add to left and right if they exist
    if idx > 0:
        values[idx - 1] += left
    if idx < len(values) - 2:
        values[idx + 2] += right
    # replace pair with 0
    
    values[idx] = 0
    del values[idx + 1]
    
    depths[idx] -= 1
    del depths[idx + 1]

In [103]:
def find_split(values):
    for ii, val in enumerate(values):
        if val >= 10:
            return ii
    return None

def split(values, depths, idx):
    to_split = values[idx]
    
    down = round(to_split / 2 - 0.1)
    up = round(to_split / 2 + 0.1)
    
    values[idx] = down
    values.insert(idx + 1, up)
    
    depths[idx] += 1
    depths.insert(idx + 1, depths[idx])

In [116]:
def reduce(values, depths):
    stuff_to_do = True
    while stuff_to_do:
        stuff_to_do = False
        exp_idx = find_explode(depths)
        split_idx = find_split(values)
        if exp_idx is not None:
    #         print("sploding")
            stuff_to_do = True
            explode(values, depths, exp_idx)
        elif split_idx is not None:
    #         print("splitting")
            stuff_to_do = True
            split(values, depths, split_idx)
    return values, depths

In [114]:
def find_pair(depths):
    for ii, pair in enumerate(zip(depths, depths[1:])):
        if pair[0] == pair[1]:
            return ii
    return None

def magnitude(values, depths):
    values = values.copy()
    depths = depths.copy()
    
    while(len(values) > 1):
        pair_idx = find_pair(depths)
        if pair_idx is not None:
            left_idx, right_idx = pair_idx, pair_idx + 1
            mag = 3 * values[left_idx] + 2 * values[right_idx]
            values[left_idx] = mag
            del values[right_idx]
            depths[left_idx] -= 1
            del depths[right_idx]
    print(values)
    return values[0]
        
        

In [138]:
def add_sfn(sfn1, sfn2):
    
    if sfn1 is None:
        return sfn2
    elif sfn2 is None:
        return sfn1
    
    v1, d1 = sfn1
    v2, d2 = sfn2
    
    return v1 + v2, [depth + 1 for depth in d1 + d2]

add_sfn(parse_sfn("[1,2]"), parse_sfn("[[3,4],5]"))

([1, 2, 3, 4, 5], [2, 2, 3, 3, 2])

In [151]:
running_sum = None
for sfn in data:
    vals_depths = parse_sfn(sfn)
    running_sum = add_sfn(vals_depths, running_sum)
    reduce(*running_sum)

magnitude(*running_sum)

[2551]


2551